# Глубинное обучение для текстовых данных, ФКН ВШЭ

## Домашнее задание 1: Text Suggestion

__Мягкий дедлайн: 24.09 23:59__   
__Жесткий дедлайн: 27.09 23:59__

### О задании

В этом задании вам предстоит реализовать систему, предлагающую удачное продолжение слова или нескольких следующих слов в режиме реального времени по типу тех, которые используются в почте или поисковой строке. За дополнительные баллы полученную систему нужно будет обернуть в пользовательский интерфейс с помощью библиотеки [reflex](https://github.com/reflex-dev/reflex) или аналогов. В этой домашке вам не придется обучать никаких моделей, мы ограничимся n-граммной генерацией.

### Структура

Это домашнее задание состоит из двух частей: основной и бонусной. В первой вам нужно будет выполнить 5 заданий, по итогам которых вы получите минимально рабочее решение. А во второй, пользуясь тем, что вы уже сделали реализовать полноценную систему подсказки текста с пользовательским интерфейсом. Во второй части мы никак не будем ограничивать вашу фантазию. Делайте что угодно, лишь бы в результате получился удобный фреймворк. Чем лучше у вас будет результат, тем больше баллов вы получите. Если будет совсем хорошо, то мы добавим бонусов сверху по своему усмотрению.

### Оценивание и штрафы

Максимально допустимая оценка за работу — 15 баллов. Сдавать задание после жесткого дедлайна нельзя. При сдачи решения после мягкого дедлайна за каждый день просрочки снимается по __одному__ баллу.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов. Весь код должен быть написан самостоятельно. Чужим кодом для пользоваться запрещается даже с указанием ссылки на источник. В разумных рамках, конечно. Взять пару очевидных строчек кода для реализации какого-то небольшого функционала можно.

Неэффективная реализация кода может негативно отразиться на оценке. Также оценка может быть снижена за плохо читаемый код.

При сдаче зададания в anytask вам будет необходимо сдать весь код, а если вы возьметесь за бонусную часть, то еще отчет и видео с демонстрацией вашего UI. За основную часть можно получить до __10-ти__ баллов, а за бонусную – до __5-ти__ баллов.

### Данные

Для получения текстовых статистик используйте датасет `emails.csv`. Вы можете найти его по [ссылке](https://disk.yandex.ru/d/ikyUhWPlvfXxCg). Он содержит более 500 тысяч электронных писем на английском языке.

In [1]:
import pandas as pd

emails = pd.read_csv('emails.csv')
len(emails)

517401

Заметьте, что данные очень грязные. В каждом письме содержится различная мета-информация, которая будет только мешать при предсказании продолжения текста.

__Задание 1 (2 балла).__ Очистите корпус текстов по вашему усмотрению и объясните свой выбор. В идеале обработанные тексты должны содержать только текст самого письма и ничего лишнего по типу ссылок, адресатов и прочих символов, которыми мы точно не хотим продолжать текст.

In [2]:
def normalize_message(message):
    message = ' '.join(message.split('\n\n')[1:])
    norm_message = ''
    for char in message:
        if char.isalpha():
            norm_message += char.lower()
        elif char in [' ', '\n', '\t'] and len(norm_message) > 0 and norm_message[-1] != ' ':
            norm_message += ' '
    return norm_message

In [3]:
emails['message_norm'] = emails['message'].map(lambda x: normalize_message(x))

In [4]:
emails[['message_norm']].to_csv('corpus.csv', index=False)

Для следующего задания вам нужно будет токенизировать текст. Для этого просто разбейте его по словам. Очевидно, итоговый результат для финального пользователя будет лучше, если ваша система также будет предлагать уместную пунктуацию. Но если вы заметите, что из-за этого падает качество самого текса, то можете удалить все небуквенные символы на этапе токенизации.

## Общая схема решения

Мы хотим сделать систему, которая будет ускорять набор текста, советуя подходящие продолжения. Для подсказки следующего слова мы будем использовать n-граммную модель. Так как n-граммная модель работает с целыми словами, а советы мы хотим давать в риал-тайме даже когда слово еще не дописано, сперва надо научиться дополнять слово до целого.

## Дополнение слова

В этой части вам предстоит реализовать метод дополнения слова до целого по его началу (префиксу). Для этого сперва необходимо научиться находить все слова, имеющие определенный префикс. Мы будем вызывать функцию поиска подходящих слов после каждой напечатанной пользователем буквы. Поэтому нам очень важно, чтобы поиск работал как можно быстрее. Простой перебор всех слов занимает $O(|V| \cdot n)$ времени, где $|V|$ – размер словаря, а $n$ – длина префикса. Мы же напишем [префиксное дерево](https://ru.wikipedia.org/wiki/Префиксное_дерево), которое позволяет искать слова не больше чем за $O(n + mk)$, где $m$ - число подходящих слов, а $k$ – длина суффикса.

__Задание 2 (2 балла).__ Допишите префиксное дерево для поиска слов по префиксу.

In [17]:
from typing import List

class PrefixTreeNode:
    def __init__(self):
        self.children: dict[str, PrefixTreeNode] = {}
        self.is_end_of_word = False

class PrefixTree:
    def __init__(self, vocabulary):
        """
        vocabulary: список всех уникальных токенов в корпусе
        """
        self.root = PrefixTreeNode()
        
        for word in vocabulary:
            self.insert(word)
            
    def insert(self, word):
        node = self.root
        for char in word:
            if char not in node.children:
                node.children[char] = PrefixTreeNode()
            node = node.children[char]
        node.is_end_of_word = True

    def search_prefix(self, prefix):
        """
        Возвращает все слова, начинающиеся на prefix
        prefix: str – префикс слова
        """
        node = self.root
        for char in prefix:
            if char not in node.children:
                return []
            node = node.children[char]

        result = []
        def dfs(curr, path):
            if curr.is_end_of_word:
                result.append(path)
            for ch, child in curr.children.items():
                dfs(child, path + ch)

        dfs(node, prefix)
        return result

In [18]:
vocabulary = ['aa', 'aaa', 'abb', 'bba', 'bbb', 'bcd']
prefix_tree = PrefixTree(vocabulary)

assert set(prefix_tree.search_prefix('a')) == set(['aa', 'aaa', 'abb'])
assert set(prefix_tree.search_prefix('bb')) == set(['bba', 'bbb'])

Теперь, когда у нас есть способ быстро находить все слова с определенным префиксом, нам нужно их упорядочить по вероятности, чтобы выбирать лучшее. Будем оценивать вероятность слова по частоте его __встречаемости в корпусе__.

__Задание 3 (2 балла).__ Допишите класс `WordCompletor`, который формирует словарь и префиксное дерево, а так же умеет находить все возможные продолжения слова вместе с их вероятностями. В этом классе вы можете при необходимости дополнительно отфильтровать слова, например, удалив все самые редкие. Постарайтесь максимально оптимизировать ваш код.

In [19]:
class WordCompletor:
    def __init__(self, corpus):
        """
        corpus: list – корпус текстов
        """
        self.freq = {}
        den = 0
        for sentence in corpus:
            for word in sentence:
                if word not in self.freq:
                    self.freq[word] = 0
                self.freq[word] += 1
                den += 1
        for word in self.freq:
            self.freq[word] = self.freq[word] / den
        self.prefix_tree = PrefixTree(self.freq.keys())

    def get_words_and_probs(self, prefix: str):
        """
        Возвращает список слов, начинающихся на prefix,
        с их вероятностями (нормировать ничего не нужно)
        """
        words, probs = [], []
        possible_words = self.prefix_tree.search_prefix(prefix)
        for w in possible_words:
            words.append(w)
            probs.append(self.freq[w])
        return words, probs

In [20]:
dummy_corpus = [
    ["aa", "ab"],
    ["aaa", "abab"],
    ["abb", "aa", "ab", "bba", "bbb", "bcd"],
]

word_completor = WordCompletor(dummy_corpus)
words, probs = word_completor.get_words_and_probs('a')
words_probs = list(zip(words, probs))
assert set(words_probs) == {('aa', 0.2), ('ab', 0.2), ('aaa', 0.1), ('abab', 0.1), ('abb', 0.1)}

## Предсказание следующих слов

Теперь, когда мы умеем дописывать слово за пользователем, мы можем пойти дальше и предожить ему следующее слово (или несколько) с учетом дописанного. Для этого мы воспользуемся n-граммной моделью.

Напомним, что вероятность последовательности для такой модели записывается по формуле
$$
P(w_1, \dots, w_T) = \prod_{i=1}^T P(w_i \mid w_{i-1}, \dots, w_{i-n}).
$$

$P(w_i \mid w_{i-1}, \dots, w_{i-n})$ оценивается по частоте встречаемости n-граммы.   

__Задание 4 (2 балла).__ Напишите класс для n-граммной модели. Никакого сглаживания добавлять не надо, мы же не хотим, чтобы модель советовала случайные слова (хоть и очень редко).

In [22]:
class NGramLanguageModel:
    def __init__(self, corpus, n):
        self.n = n
        self.freq = {}
        den = {}
        for sentence in corpus:
            for i in range(max(len(sentence) - n, 0)):
                if tuple(sentence[i:i + n]) not in self.freq:
                    self.freq[tuple(sentence[i:i + n])] = {}
                if sentence[i + n] not in self.freq[tuple(sentence[i:i + n])]:
                    self.freq[tuple(sentence[i:i + n])][sentence[i + n]] = 0
                self.freq[tuple(sentence[i:i + n])][sentence[i + n]] += 1
                
                if tuple(sentence[i:i + n]) not in den:
                    den[tuple(sentence[i:i + n])] = 0
                den[tuple(sentence[i:i + n])] += 1
                
        for sent in self.freq:
            for word in self.freq[sent]:
                self.freq[sent][word] = self.freq[sent][word] / den[sent]

    def get_next_words_and_probs(self, prefix: list):
        """
        Возвращает список слов, которые могут идти после prefix,
        а так же список вероятностей этих слов
        """

        next_words, probs = [], []
        prefix = prefix[-self.n:]
        
        if tuple(prefix) not in self.freq:
            return next_words, probs
        
        for word in self.freq[tuple(prefix)]:
            next_words.append(word)
            probs.append(self.freq[tuple(prefix)][word])

        return next_words, probs

In [23]:
dummy_corpus = [
    ['aa', 'aa', 'aa', 'aa', 'ab'],
    ['aaa', 'abab'],
    ['abb', 'aa', 'ab', 'bba', 'bbb', 'bcd']
]

n_gram_model = NGramLanguageModel(corpus=dummy_corpus, n=2)

next_words, probs = n_gram_model.get_next_words_and_probs(['aa', 'aa'])
words_probs = list(zip(next_words, probs))

assert set(words_probs) == {('aa', 2/3), ('ab', 1/3)}

Отлично, мы теперь можем объединить два метода в автоматический дописыватель текстов: первый будет дополнять слово, а второй – предлагать продолжения. Хочется, чтобы предлагался список возможных продолжений, из который пользователь сможет выбрать наиболее подходящее. Самое сложное тут – аккуратно выбирать, что показывать, а что нет.   

__Задание 5 (2 балла).__ В качестве первого подхода к снаряду реализуйте метод, возвращающий всегда самое вероятное продолжение жадным способом. После этого можно добавить опцию генерации нескольких вариантов продолжений, что сделает метод гораздо лучше.

In [24]:
import numpy as np

In [44]:
import numpy as np

class TextSuggestion:
    def __init__(self, word_completor, n_gram_model):
        self.word_completor = word_completor
        self.n_gram_model = n_gram_model

    def suggest_text(self, text, n_words=3, n_texts=1):
        """
        Возвращает возможные варианты продолжения текста (по умолчанию только один)
        
        text: строка или список слов – написанный пользователем текст
        n_words: число слов, которые дописывает n-граммная модель
        n_texts: число возвращаемых продолжений (пока что только одно)
        
        return: list[list[srt]] – список из n_texts списков слов, по 1 + n_words слов в каждом
        Первое слово – это то, которое WordCompletor дополнил до целого.
        """
        if len(text) == 0:
            return [], []
        
        if isinstance(text, str) and text[-1] == ' ':
            text = text.split()
            
        def normalize_message(message):
            norm_message = ''
            for char in message:
                if char.isalpha():
                    norm_message += char.lower()
                elif char in [' ', '\n', '\t'] and len(norm_message) > 0 and norm_message[-1] != ' ':
                    norm_message += ' '
            return norm_message
        
        if isinstance(text, str):
            text = normalize_message(text)
        else:
            text = normalize_message(' '.join(text)).split()

        if isinstance(text, str):
            words, probs = self.word_completor.get_words_and_probs(text.split()[-1])
            if len(words) == 0:
                return [], [text]
            else:
                text = text.split()[:-1]
        else:
            words, probs = self.n_gram_model.get_next_words_and_probs(text)
            if len(words) == 0:
                return [], [' '.join(text)]
        
        best_words = [words[i] for i in np.argsort(probs)[::-1][:n_texts]]
        suggestions = []
        texts = []
        for word in best_words:
            suggestions_word = [word]
            text_word = text + [word]
            for _ in range(n_words - 1):
                words, probs = self.n_gram_model.get_next_words_and_probs(text_word)
                if len(words) == 0:
                    break
                next_word = words[np.argmax(probs)]
                suggestions_word.append(next_word)
                text_word.append(next_word)
            suggestions.append(suggestions_word)
            texts.append(text_word)
        
        return [[text[-1]] + suggestions[0]] if len(suggestions) < n_words + 1 else suggestions

In [46]:
dummy_corpus = [
    ['aa', 'aa', 'aa', 'aa', 'ab'],
    ['aaa', 'abab'],
    ['abb', 'aa', 'ab', 'bba', 'bbb', 'bcd']
]

word_completor = WordCompletor(dummy_corpus)
n_gram_model = NGramLanguageModel(corpus=dummy_corpus, n=2)
text_suggestion = TextSuggestion(word_completor, n_gram_model)

assert text_suggestion.suggest_text(['aa', 'aa'], n_words=3, n_texts=1) == [['aa', 'aa', 'aa', 'aa']]
assert text_suggestion.suggest_text(['abb', 'aa', 'ab'], n_words=2, n_texts=1) == [['ab', 'bba', 'bbb']]

In [49]:
text_suggestion = TextSuggestion(word_completor, n_gram_model)

## Бонусная часть: Добавляем UI

Запускать ячейки в юпитере – это хорошо, но будет лучше, если вашим решением действительно можно будет пользоваться. Для этого вам предлагается добавить полноценных User Interface. Мы рекомендуем использовать для этого [reflex](https://github.com/reflex-dev/reflex). Это Python библиотека для создания web-интерфейсом с очень богатым функционалом.

Ваша задача – сделать поле для текстового ввода, при наборе текста в котором будут появляться подсказки в реальном времени. Продумайте, как пользователь будет выбирать подсказки, сколько продолжений рекомендавать и так далее. В общем, должно получиться красиво и удобно. В этой части вы можете модифицировать все классы по своему усмотрению и добавлять любые эвристики. Если нужно, то дополнительно обрабатывать текст и вообще делать все, что считаете нужным. 

За это задание можно получить до __5-ти бонусных баллов__ в зависимости о того, насколько хорошо и удобно у вас получилось. При сдаче задания прикрепите небольшой __отчет__ (полстраницы) с описанием вашей системы, а также __видео__ (1-2 минуты) с демонстрацией работы интерфейса.

Мы настоятельно рекомендуем вам оформить код в проект, а не писать в ноутбуке. Но если вам очень хочется писать тут, то хотя бы не меняйте код в предыдущих заданиях, чтобы его можно было нормально оценивать.